In [0]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import *
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
! pip install kaggle --upgrade
! mkdir -p ~/.kaggle/
! cp 'gdrive/My Drive/Colab Notebooks/kaggle/kaggle.json' ~/.kaggle/
! mkdir imdb

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.2)


In [4]:
! kaggle competitions download -c comp-551-imbd-sentiment-classification -p imdb

 91% 18.0M/19.9M [00:00<00:00, 57.0MB/s]
100% 19.9M/19.9M [00:00<00:00, 78.9MB/s]
 85% 24.0M/28.1M [00:00<00:00, 42.7MB/s]
100% 28.1M/28.1M [00:00<00:00, 93.3MB/s]
  0% 0.00/184k [00:00<?, ?B/s]
100% 184k/184k [00:00<00:00, 56.2MB/s]


In [0]:
! unzip -q imdb/test.zip -d imdb/
! unzip -q imdb/train.zip -d imdb/


In [0]:
train = pd.DataFrame(columns=['Id', 'Text', 'Target'])
label = {'pos': 1, 'neg':0}
for target in label.keys():
  for file in os.listdir('imdb/train/%s/'%(target)):
    id = file.replace('.txt', '')
    with open('imdb/train/%s/%s'%(target, file)) as fp:
      train = train.append({'Id': id, 'Text': fp.read(), 'Target': label[target]}, ignore_index=True)
train.set_index('Id', inplace=True)

In [0]:
test = pd.DataFrame(columns=['Id', 'Text'])

for file in os.listdir('imdb/test/'):
  id = file.replace('.txt', '')
  with open('imdb/test/%s'%(file)) as fp:
      test = test.append({'Id': id, 'Text': fp.read()}, ignore_index=True)
test.set_index('Id', inplace=True)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(strip_accents='unicode', lowercase=True
#                              , stop_words='english'
                             ,ngram_range=(1,2)
                            )

full_text = list(train.iloc[:,0].values) + list(test.iloc[:,0].values)
vectorizer.fit(full_text)
X_train = vectorizer.transform(train.iloc[:,0].values)
X_test = vectorizer.transform(test.iloc[:,0].values)
y = train.iloc[:,1].values.astype(int)

In [20]:
X_train.shape, X_test.shape

((25000, 2466212), (25000, 2466212))

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

parameters_mnb = {'alpha':[0, 1, 1.5, 2, 2.5, 3]}
parameters_svm = {'C':[1], 'kernel': ['linear', 'rbf']}
parameters_regression = {'penalty':['l2']
                         ,'C': [1,2,3]
                         #'fit_intercept':[True, False], 'max_iter':[1000]
                        ,'solver':['sag', 'saga']}

scoring = 'f1'

naive_bayes = MultinomialNB()
svm = SVC()
regression = LogisticRegression()
                  
clf = GridSearchCV(svm, parameters_svm, cv=25, scoring=scoring, n_jobs=-1, verbose=3)
clf.fit(X_train, y)

Fitting 25 folds for each of 2 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [0]:
clf.cv_results_

In [0]:
clf.best_params_, clf.best_score_

In [0]:
model = clf.best_estimator_
y_pred = model.predict(X_test)
test['Category'] = y_pred

In [0]:
submission = test.drop(columns='Text')
submission.to_csv('submission.csv')

In [0]:
! kaggle competitions submit -c comp-551-imbd-sentiment-classification -f submission.csv -m "dummy submission"

100% 184k/184k [00:01<00:00, 143kB/s]
Successfully submitted to COMP 551 - IMBD Sentiment Classification